<summary>Table of Contents</summary>

- [1. TimeLLM](#1-timellm)
- [2. TimeLLM](#2-timellm-336)

Results for TimeLLM. The first one is default input length 512, the second one: 336.

In [1]:
import os
import pandas as pd
import subprocess
import time
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

In [2]:
# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE']
num_cols = [5]
seq_lens = [512]

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 1. TimeLLM

In [3]:
log_dir = f"logs/timellm/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
model = "TimeLLM"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_DE.log"

# Parameters for tuning
lr = 0.001 # 10^-3 
train_epochs = 20
patience = 5
d_model = 16
d_ff = 64
batch_size = 32

# List to store the results
timellm_results = []

In [4]:
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)
            
            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --multi_gpu --num_processes=4 --num_machines 1 --dynamo_backend "no"  --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience {patience} \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

[2024-11-16 13:35:46,795] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-16 13:35:46,816] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-16 13:35:46,822] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-16 13:35:46,822] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-16 13:35:47,677] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-16 13:35:47,677] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-16 13:35:47,677] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-16 13:35:47,677] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-16 13:35:47,677] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
train 143885

In [5]:
timellm_results

[{'Country': 'DE',
  'Pred_len': 24,
  'MSE': 0.021544765681028366,
  'RMSE': 0.14678135514259338,
  'MAE': 0.09620458632707596},
 {'Country': 'DE',
  'Pred_len': 96,
  'MSE': 0.0346515066921711,
  'RMSE': 0.18614915013313293,
  'MAE': 0.12807267904281616},
 {'Country': 'DE',
  'Pred_len': 168,
  'MSE': 0.03746809437870979,
  'RMSE': 0.19356676936149597,
  'MAE': 0.1385875940322876}]

In [5]:
# Dynamic variables
pred_lens = [24]
countries = ['FR']
num_cols = [3]
seq_lens = [168]
train_epochs = 30

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [6]:
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)
            
            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --multi_gpu --num_processes=4 --num_machines 1 --dynamo_backend "no"  --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience {patience} \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: FR ===


=== Starting experiments for pred_len: 24 ===

[2024-11-14 13:57:59,560] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-14 13:57:59,568] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-14 13:57:59,569] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-14 13:57:59,571] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-14 13:58:03,477] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-14 13:58:03,477] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-14 13:58:03,477] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-14 13:58:03,477] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-14 13:58:03,477] [INFO] [comm.py:637:init_distributed] cdb=None
train 86835


In [7]:
timellm_results

[{'Country': 'FR',
  'Pred_len': 24,
  'MSE': 0.01039215736091137,
  'RMSE': 0.10194192826747894,
  'MAE': 0.05944359302520752}]

In [11]:
res = [{'Country': 'DE',
  'Pred_len': 24,
  'MSE': 0.021544765681028366,
  'RMSE': 0.14678135514259338,
  'MAE': 0.09620458632707596},
 {'Country': 'DE',
  'Pred_len': 96,
  'MSE': 0.0346515066921711,
  'RMSE': 0.18614915013313293,
  'MAE': 0.12807267904281616},
 {'Country': 'DE',
  'Pred_len': 168,
  'MSE': 0.03746809437870979,
  'RMSE': 0.19356676936149597,
  'MAE': 0.1385875940322876},
 {'Country': 'GB',
  'Pred_len': 24,
  'MSE': 0.02595963142812252,
  'RMSE': 0.16111992299556732,
  'MAE': 0.10740001499652863},
 {'Country': 'GB',
  'Pred_len': 96,
  'MSE': 0.04068015143275261,
  'RMSE': 0.20169320702552795,
  'MAE': 0.14278638362884521},
 {'Country': 'GB',
  'Pred_len': 168,
  'MSE': 0.043486617505550385,
  'RMSE': 0.20853444933891296,
  'MAE': 0.14944525063037872},
 {'Country': 'ES',
  'Pred_len': 24,
  'MSE': 0.010894566774368286,
  'RMSE': 0.10437703877687454,
  'MAE': 0.0666496753692627},
 {'Country': 'ES',
  'Pred_len': 96,
  'MSE': 0.02007182687520981,
  'RMSE': 0.14167506992816925,
  'MAE': 0.09592236578464508},
 {'Country': 'ES',
  'Pred_len': 168,
  'MSE': 0.021907182410359383,
  'RMSE': 0.1480107456445694,
  'MAE': 0.10050459951162338},
 {'Country': 'FR',
  'Pred_len': 24,
  'MSE': 0.01039215736091137,
  'RMSE': 0.10194192826747894,
  'MAE': 0.05944359302520752},
 {'Country': 'FR',
  'Pred_len': 96,
  'MSE': 0.0196530781686306,
  'RMSE': 0.14018943905830383,
  'MAE': 0.08574584871530533},
 {'Country': 'FR',
  'Pred_len': 168,
  'MSE': 0.02125428430736065,
  'RMSE': 0.14578849077224731,
  'MAE': 0.09115074574947357},
 {'Country': 'IT',
  'Pred_len': 24,
  'MSE': 0.01045023463666439,
  'RMSE': 0.1022263914346695,
  'MAE': 0.06180146336555481},
 {'Country': 'IT',
  'Pred_len': 96,
  'MSE': 0.01832490973174572,
  'RMSE': 0.13536952435970306,
  'MAE': 0.08470133692026138},
 {'Country': 'IT',
  'Pred_len': 168,
  'MSE': 0.01952982507646084,
  'RMSE': 0.13974915444850922,
  'MAE': 0.08981484919786453}]

In [12]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/timellm'

if not os.path.exists(path):
    os.makedirs(path)

timellm_df = convert_results_into_df(res, if_loss_fnc=False, itr=1)

# Final DF
timellm_df.columns = pd.MultiIndex.from_product([['TimeLLM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
timellm_df.to_csv(os.path.join(path, 'timellm.csv'))
timellm_df.round(4)

Model            TimeLLM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0215  0.1468  0.0962
        96        0.0347  0.1861  0.1281
        168       0.0375  0.1936  0.1386
GB      24        0.0260  0.1611  0.1074
        96        0.0407  0.2017  0.1428
        168       0.0435  0.2085  0.1494
ES      24        0.0109  0.1044  0.0666
        96        0.0201  0.1417  0.0959
        168       0.0219  0.1480  0.1005
FR      24        0.0104  0.1019  0.0594
        96        0.0197  0.1402  0.0857
        168       0.0213  0.1458  0.0912
IT      24        0.0105  0.1022  0.0618
        96        0.0183  0.1354  0.0847
        168       0.0195  0.1397  0.0898

# 2. TimeLLM 336

Sequence length 336.

In [ ]:
log_dir = f"logs/timellm/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
model = "TimeLLM"
seq_len = 336

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_336.log"

# Parameters for tuning,but default
lr = 0.001 # 10^-3 
train_epochs = 20
d_model = 16
d_ff = 64
batch_size = 32

In [ ]:
# List to store the results
timellm_results = []

start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --patience 5 \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)[0]
            mse, rmse, mae, _ = iteration_metrics
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

train 143885
val 31085
test 31085
[2024-11-02 04:05:41,780] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-11-02 04:05:42,950] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-11-02 04:05:42,950] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-11-02 04:05:42,950] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-11-02 04:05:43,048] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.43, master_port=29500
[2024-11-02 04:05:43,048] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-11-02 04:05:43,713] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled

In [ ]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/timellm'

if not os.path.exists(path):
    os.makedirs(path)

timellm_df = convert_results_into_df(timellm_results, if_loss_fnc=False, itr=1)

# Final DF
timellm_df.columns = pd.MultiIndex.from_product([['TimeLLM/336'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
timellm_df.to_csv(os.path.join(path, 'timellm_336.csv'))
timellm_df.round(4)

Model            TimeLLM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0217  0.1474  0.0920
        96        0.0395  0.1989  0.1324
        168       0.0395  0.1988  0.1348
GB      24        0.0252  0.1588  0.1012
        96        0.0437  0.2091  0.1434
        168       0.0469  0.2166  0.1487
ES      24        0.0108  0.1039  0.0672
        96        0.0198  0.1409  0.0941
        168       0.0217  0.1472  0.0970
FR      24        0.0104  0.1017  0.0586
        96        0.0186  0.1365  0.0824
        168       0.0205  0.1430  0.0878
IT      24        0.0104  0.1022  0.0607
        96        0.0188  0.1370  0.0839
        168       0.0204  0.1428  0.0892